# ДЗ. Урок 11. Модель Transformer

1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)

In [10]:
!pip install transformers
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import io
import re

### 1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)

In [ ]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-08-05 08:54:16--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  23.5MB/s    in 0.6s    

2022-08-05 08:54:16 (23.5 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]



In [ ]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [12]:
path_to_file = "rus-eng/rus.txt"

In [13]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [14]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [15]:
en, ru = create_dataset(path_to_file, None)
print(en[20])
print(ru[20])

<start> duck ! <end>
<start> пригнись ! <end>


In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en", do_lower_case=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

In [24]:
def translate(sentence):

  inputs = tokenizer.encode(sentence, return_tensors="pt")
  outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

  result = tokenizer.decode(outputs[0])

  return result

In [29]:
# хаотично беру предложения из корпуса с прошлого (10) домашнего задания.

for i in [5, 20, 50, 100, 256, 523, 526, 548, 4536, 62458, 46358, 1596]:
    print(ru[i], translate(ru[i]))

<start> хай . <end> <start> hey. <end
<start> пригнись ! <end> <start> get down! <end
<start> продолжайте . <end> <start> continue. <end
<start> съешьте это . <end> <start> eat this. <end
<start> сложи его . <end> <start> folding it. <end
<start> не ездите . <end> <start> do not go. <end
<start> едем дальше . <end> <start> moving on. <end
<start> убирайся ! <end> #start> get out! #end
<start> я пакую вещи . <end> <start> I pack things. <end
<start> том слишком много говорит . <end> <start> volume speaks too much. <end
<start> спасибо за ужин . <end> #start> Thank you for dinner. <end
<start> так оно и есть . <end> <start> so it is. <end


На мой взгляд предобученная трансформерная архитектура справилась с переводом очень неплохо, за исключением этой строки "<start> том слишком много говорит . <end> <start> volume speaks too much. <end" - это надо с именами собственными поработать.